# Task 4: Pretrained Language Models (BERT, GPT, T5)

In [2]:
pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing 

# BERT

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

#IMDB
dataset = load_dataset("imdb")

#BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

tokenized_ds = dataset.map(tokenize, batched=True)
tokenized_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])

model_BERT = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {"accuracy": accuracy_score(labels, preds)}

training_args = TrainingArguments(
    output_dir="./bert-imdb",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    logging_dir="./logs",
    logging_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1,
    report_to="none"
)


trainer = Trainer(
    model=model_BERT,
    args=training_args,
    train_dataset=tokenized_ds["train"].shuffle(seed=42).select(range(5000)),  # ускорим обучение
    eval_dataset=tokenized_ds["test"].select(range(1000)),
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.394400,0.425322,0.810000
2,0.181100,0.552681,0.824000
3,0.094300,0.618094,0.873000
4,0.022300,0.754065,0.859000


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1252, training_loss=0.1600307113827227, metrics={'train_runtime': 672.8547, 'train_samples_per_second': 29.724, 'train_steps_per_second': 1.861, 'total_flos': 2631110553600000.0, 'train_loss': 0.1600307113827227, 'epoch': 4.0})

In [6]:
eval_result = trainer.evaluate()
print(eval_result)

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.6180936098098755, 'eval_accuracy': 0.873, 'eval_runtime': 10.3512, 'eval_samples_per_second': 96.607, 'eval_steps_per_second': 6.086, 'epoch': 4.0}


# GPT

In [2]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score

# Загружаем датасет Shakespeare
dataset = load_dataset("tiny_shakespeare")
print(dataset)


2025-05-06 22:35:57.280579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746570957.509617      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746570957.572905      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


README.md:   0%|          | 0.00/6.10k [00:00<?, ?B/s]

tiny_shakespeare.py:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

The repository for tiny_shakespeare contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/tiny_shakespeare.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1
    })
})


In [3]:
from transformers import GPT2Tokenizer
from datasets import load_dataset
import itertools

# === 1. Загрузка токенизатора ===
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT2 не имеет pad_token по умолчанию

# === 2. Загрузка датасета (предположим, уже есть текстовая колонка 'text') ===
# dataset = load_dataset("path_to_shakespeare_dataset")
# Пример: dataset = load_dataset("tiny_shakespeare")  # если есть такой

# === 3. Токенизация ===
def tokenize_function(example):
    return tokenizer(example["text"], return_attention_mask=False)

tokenized = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

# === 4. Удаление всех колонок кроме input_ids ===
tokenized = tokenized.remove_columns(
    [col for col in tokenized["train"].column_names if col != "input_ids"]
)

# === 5. Группировка в блоки фиксированной длины ===
block_size = 250

def group_texts(examples):
    concatenated = list(itertools.chain.from_iterable(examples["input_ids"]))
    total_length = (len(concatenated) // block_size) * block_size
    input_ids = [
        concatenated[i : i + block_size] 
        for i in range(0, total_length, block_size)
    ]
    return {
        "input_ids": input_ids,
        "labels": input_ids.copy(),  # language modeling
    }

lm_dataset = tokenized.map(group_texts, batched=True)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (301966 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [4]:
from transformers import GPT2LMHeadModel

model_GPT = GPT2LMHeadModel.from_pretrained("gpt2")
model_GPT.resize_token_embeddings(len(tokenizer))


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-shakespeare",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    num_train_epochs=20,
    save_strategy="no",
    logging_steps=200,
    report_to="none"
)


from transformers import Trainer

trainer = Trainer(
    model=model_GPT,
    args=training_args,
    train_dataset=lm_dataset["train"]
)

trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
200,1.887600
400,1.775500
600,1.710200
800,1.653000
1000,1.651000
1200,1.622400
1400,1.562100
1600,1.535600
1800,1.515300
2000,1.477700


TrainOutput(global_step=6040, training_loss=1.4315940073783824, metrics={'train_runtime': 1212.7101, 'train_samples_per_second': 19.906, 'train_steps_per_second': 4.981, 'total_flos': 3079877760000000.0, 'train_loss': 1.4315940073783824, 'epoch': 20.0})

In [6]:
from transformers import pipeline

generator = pipeline("text-generation", model=model_GPT, tokenizer=tokenizer)

seeds = [
    "To be, or not to be",
    "O Romeo, Romeo! wherefore art thou",
    "The night is dark and",
    "What light through yonder window breaks?",
    "I am the king of",
    "Thou art more lovely and more temperate",
]

results = []

for seed in seeds:
    out = generator(seed, max_length=100, num_return_sequences=1)[0]["generated_text"]
    results.append((seed, out))



for seed, gen in results:
    print(f"Seed: {seed}")
    print(f"Generated:\n{gen}")
    print("-" * 50)

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Seed: To be, or not to be
Generated:
To be, or not to be; the answer is as follows:
I do not want thy word, but to chide at
the thing thou hast ignorant of; being a soldier,
I could not show him the book, or give him
a map of the place; nor wouldst give him any thing, but
sneak'd me in my place; which I think is nothing, having
such ignorance as to do either thing with
no matter.

ANG
--------------------------------------------------
Seed: O Romeo, Romeo! wherefore art thou
Generated:
O Romeo, Romeo! wherefore art thou?

ROMEO:
Yond, that I may kiss thee; wherefore be I?

ROMEO:
Nay, thou knowest me best: and wherefore art thou
wise?

JULIET:
Marry, I know, to be no better.

ROMEO:
I have but one eye more; the other is mute:
For I never met a sweeter ear,
--------------------------------------------------
Seed: The night is dark and
Generated:
The night is dark and I cannot sleep:
Lovers, I have heard, that never in the world
Have I had a better nap than this.

BENVOLIOLE:
This is the

# T5

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model_t5 = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
text = "translate English to French: The weather is nice today"

inputs = tokenizer(text, return_tensors="pt", padding=True)
outputs = model_t5.generate(**inputs, max_length=40)

print("Translation:", tokenizer.decode(outputs[0], skip_special_tokens=True))


Translation: Le temps est agréable aujourd'hui


In [9]:
article = """summarize: Machine learning is a method of data analysis that automates analytical model building.
It is a branch of artificial intelligence based on the idea that systems can learn from data,
identify patterns and make decisions with minimal human intervention."""

inputs = tokenizer(article, return_tensors="pt", padding=True, truncation=True, max_length=512)
outputs = model_t5.generate(**inputs, max_length=40)

print("Summary:", tokenizer.decode(outputs[0], skip_special_tokens=True))


Summary: machine learning is a branch of artificial intelligence based on the idea that systems can learn from data, identify patterns and make decisions with minimal human intervention.


In [10]:
sentences = [
    "translate English to German: I love natural language processing.",
    "translate English to German: This model is very powerful.",
    "translate English to German: Have a nice day!"
]

for s in sentences:
    inputs = tokenizer(s, return_tensors="pt")
    outputs = model_t5.generate(**inputs, max_length=40)
    print(f"{s.split(':')[1].strip()} → {tokenizer.decode(outputs[0], skip_special_tokens=True)}")


I love natural language processing. → Ich liebe natürliche Sprachenverarbeitung.
This model is very powerful. → Dieses Modell ist sehr leistungsstark.
Have a nice day! → Haben Sie einen schönen Tag!
